In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch.optim import lr_scheduler
import torchmetrics
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np

import collections
import pandas as pd
import json
from tqdm.auto import tqdm, trange

from datasets import load_metric
import datasets
from transformers import AutoConfig, AutoTokenizer, BertModel, RobertaModel
from transformers import BertForSequenceClassification
from transformers import TrainingArguments, Trainer

import matplotlib.pyplot as plt


In [2]:
import sys
import os
sys.path.append(f'{os.getcwd()}/SentEval')
PATH_TO_DATA = f'{os.getcwd()}/SentEval/data'

# Import SentEval
import senteval

In [3]:
# https://github.com/huggingface/transformers/issues/5486
# os.environ["TOKENIZERS_PARALLELISM"] = "false-"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
with open('tasks.json', 'r') as f:
    tasks = json.load(f)
tasks

{'CrowdFlower': 13,
 'DailyDialog': 7,
 'EmoBank_Valence': 1,
 'EmoBank_Arousal': 1,
 'EmoBank_Dominance': 1,
 'HateOffensive': 3,
 'PASTEL_age': 8,
 'PASTEL_country': 2,
 'PASTEL_education': 10,
 'PASTEL_ethnic': 10,
 'PASTEL_gender': 3,
 'PASTEL_politics': 3,
 'PASTEL_tod': 5,
 'SARC': 2,
 'SarcasmGhosh': 2,
 'SentiTreeBank': 1,
 'ShortHumor': 2,
 'ShortJokeKaggle': 2,
 'ShortRomance': 2,
 'StanfordPoliteness': 1,
 'TroFi': 2,
 'VUA': 2}

In [6]:
base_model = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [7]:
class MyDataset(Dataset): 
    # currently it's a Mapping-style dataset. Not sure if a Iterable-style dataset will be better
    def __init__(self, tsv_file):
        self.df = pd.read_csv(tsv_file, sep='\t')
        self.df = self.df.dropna()
        self.df = self.df.reset_index(drop=True)
        self.encodings = tokenizer(self.df['text'].tolist(), truncation=True, padding=True, max_length=128)
        if self.df['label'].dtype == 'float64':
            self.df['label'] = self.df['label'].astype('float32')
        self.labels = self.df['label'].tolist()
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item


In [8]:
pearsonr = load_metric("pearsonr")
spearmanr = load_metric("spearmanr")

In [9]:
# GPU memory usage: 6617 - 6680mb with bs 32
# bs 64 gives OOM
# bs 48 GPU memory 7894
batch_size = 32

In [ ]:
def single_train(task):
    torch.cuda.empty_cache()
    model = None
    trainer = None 
    
    num_labels = tasks[task]
    train_dataset = MyDataset(f'./processed/train/{task}.tsv')
    test_dataset = MyDataset(f'./processed/test/{task}.tsv')
    valid_dataset = MyDataset(f'./processed/dev/{task}.tsv')
    
    
    singletaskbert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels) 
    training_args = TrainingArguments(
        output_dir=f'./results/baselines/{task}',          # output directory
        num_train_epochs=5,              # total number of training epochs
        per_device_train_batch_size=batch_size,  # batch size per device during training
        per_device_eval_batch_size=batch_size,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir=f'./results/baselines/{task}/logs',            # directory for storing logs
        load_best_model_at_end=False,     # load the best model when finished training (default metric is loss)
        # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
        logging_first_step = True, 
#         logging_steps=500,               # log & save weights each logging_steps
#         save_steps=500,
        evaluation_strategy="epoch",     # evaluate each `logging_steps`
        save_strategy='epoch'
    )
    from sklearn.metrics import precision_recall_fscore_support
    
    if num_labels == 1:
        def compute_metrics(pred):
            predictions, labels = pred
            rmse = mean_squared_error(labels, predictions, squared=False)
            return {"rmse": rmse}
    elif num_labels == 2:
        def compute_metrics(pred):
            labels = pred.label_ids
            preds = pred.predictions.argmax(-1)
            precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
            acc = accuracy_score(labels, preds)
            return {
                'accuracy': acc,
                'f1': f1,
                'precision': precision,
                'recall': recall
            }
    else:
        def compute_metrics(pred):
            labels = pred.label_ids
            preds = pred.predictions.argmax(-1)
            precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
            acc = accuracy_score(labels, preds)
            return {
                'accuracy': acc,
                'f1': f1,
                'precision': precision,
                'recall': recall
            }

    
    trainer = Trainer(
        model=singletaskbert,   # the instantiated Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=valid_dataset,          # evaluation dataset
#         test_dataset=test_dataset,            # test dataset
        compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    )
    trainer.train()

In [10]:
for task in tasks:
    if os.path.isdir(f'./results/baselines/{task}'):
        print(f'skip {task}')
        continue
    torch.cuda.empty_cache()
    model = None
    trainer = None 
    
    num_labels = tasks[task]
    train_dataset = MyDataset(f'./processed/train/{task}.tsv')
    test_dataset = MyDataset(f'./processed/test/{task}.tsv')
    valid_dataset = MyDataset(f'./processed/dev/{task}.tsv')
    
    
    singletaskbert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels) 
    training_args = TrainingArguments(
        output_dir=f'./results/baselines/{task}',          # output directory
        num_train_epochs=5,              # total number of training epochs
        per_device_train_batch_size=batch_size,  # batch size per device during training
        per_device_eval_batch_size=batch_size,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir=f'./results/baselines/{task}/logs',            # directory for storing logs
        load_best_model_at_end=False,     # load the best model when finished training (default metric is loss)
        # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
        logging_first_step = True, 
#         logging_steps=500,               # log & save weights each logging_steps
#         save_steps=500,
        evaluation_strategy="epoch",     # evaluate each `logging_steps`
        save_strategy='epoch'
    )
    from sklearn.metrics import precision_recall_fscore_support
    
    if num_labels == 1:
        def compute_metrics(pred):
            predictions, labels = pred
            rmse = mean_squared_error(labels, predictions, squared=False)
            return {"rmse": rmse}
    elif num_labels == 2:
        def compute_metrics(pred):
            labels = pred.label_ids
            preds = pred.predictions.argmax(-1)
            precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
            acc = accuracy_score(labels, preds)
            return {
                'accuracy': acc,
                'f1': f1,
                'precision': precision,
                'recall': recall
            }
    else:
        def compute_metrics(pred):
            labels = pred.label_ids
            preds = pred.predictions.argmax(-1)
            precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
            acc = accuracy_score(labels, preds)
            return {
                'accuracy': acc,
                'f1': f1,
                'precision': precision,
                'recall': recall
            }

    
    trainer = Trainer(
        model=singletaskbert,   # the instantiated Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=valid_dataset,          # evaluation dataset
#         test_dataset=test_dataset,            # test dataset
        compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    )
    trainer.train()

skip CrowdFlower


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.394200,0.274151,0.897385,0.346008,0.387485,0.334119
2,0.318800,0.267936,0.900483,0.393077,0.477817,0.369968
3,0.247600,0.305294,0.889330,0.419818,0.548605,0.415887
4,0.178000,0.355949,0.888710,0.422777,0.545004,0.415109
5,0.126100,0.417322,0.891436,0.408869,0.473858,0.391538


***** Running Evaluation *****
  Num examples = 8069
  Batch size = 32
/home/joey/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./results/baselines/DailyDialog/checkpoint-2725
Configuration saved in ./results/baselines/DailyDialog/checkpoint-2725/config.json
Model weights saved in ./results/baselines/DailyDialog/checkpoint-2725/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 8069
  Batch size = 32
Saving model checkpoint to ./results/baselines/DailyDialog/checkpoint-5450
Configuration saved in ./results/baselines/DailyDialog/checkpoint-5450/config.json
Model weights saved in ./results/baselines/DailyDialog/checkpoint-5450/pytorch_model.bin
***** Running Evaluation *****
 

Epoch,Training Loss,Validation Loss,Rmse
1,0.537000,0.006395,0.079971
2,0.037500,0.009230,0.096074
3,0.037500,0.005136,0.071664
4,0.004000,0.005653,0.075187
5,0.004000,0.005192,0.072059


...

***** Running Evaluation *****
  Num examples = 530
  Batch size = 32
Saving model checkpoint to ./results/baselines/StanfordPoliteness/checkpoint-309
Configuration saved in ./results/baselines/StanfordPoliteness/checkpoint-309/config.json
Model weights saved in ./results/baselines/StanfordPoliteness/checkpoint-309/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 530
  Batch size = 32
Saving model checkpoint to ./results/baselines/StanfordPoliteness/checkpoint-618
Configuration saved in ./results/baselines/StanfordPoliteness/checkpoint-618/config.json
Model weights saved in ./results/baselines/StanfordPoliteness/checkpoint-618/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 530
  Batch size = 32
Saving model checkpoint to ./results/baselines/StanfordPoliteness/checkpoint-927
Configuration saved in ./results/baselines/StanfordPoliteness/checkpoint-927/config.json
Model weights saved in ./results/baselines/StanfordPoliteness/checkpoint-927/pytorch_model

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.716800,0.666300,0.548571,0.048193,1.000000,0.024691
2,0.716800,0.618478,0.645714,0.630952,0.609195,0.654321
3,0.716800,0.576512,0.714286,0.671053,0.718310,0.629630
4,0.716800,0.819517,0.651429,0.590604,0.647059,0.543210
5,0.478200,0.777154,0.742857,0.676259,0.810345,0.580247


***** Running Evaluation *****
  Num examples = 175
  Batch size = 32
Saving model checkpoint to ./results/baselines/TroFi/checkpoint-105
Configuration saved in ./results/baselines/TroFi/checkpoint-105/config.json
Model weights saved in ./results/baselines/TroFi/checkpoint-105/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 175
  Batch size = 32
Saving model checkpoint to ./results/baselines/TroFi/checkpoint-210
Configuration saved in ./results/baselines/TroFi/checkpoint-210/config.json
Model weights saved in ./results/baselines/TroFi/checkpoint-210/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 175
  Batch size = 32
Saving model checkpoint to ./results/baselines/TroFi/checkpoint-315
Configuration saved in ./results/baselines/TroFi/checkpoint-315/config.json
Model weights saved in ./results/baselines/TroFi/checkpoint-315/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 175
  Batch size = 32
Saving model checkpoint to ./results/basel

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.701700,0.401460,0.822955,0.650602,0.697674,0.609481
2,0.505300,0.349731,0.851038,0.720183,0.731935,0.708804
3,0.334800,0.445463,0.863858,0.721598,0.807263,0.652370
4,0.170300,0.629261,0.860806,0.726619,0.774936,0.683973
5,0.073000,0.754388,0.862637,0.736225,0.765854,0.708804


***** Running Evaluation *****
  Num examples = 1638
  Batch size = 32
Saving model checkpoint to ./results/baselines/VUA/checkpoint-474
Configuration saved in ./results/baselines/VUA/checkpoint-474/config.json
Model weights saved in ./results/baselines/VUA/checkpoint-474/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1638
  Batch size = 32
Saving model checkpoint to ./results/baselines/VUA/checkpoint-948
Configuration saved in ./results/baselines/VUA/checkpoint-948/config.json
Model weights saved in ./results/baselines/VUA/checkpoint-948/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1638
  Batch size = 32
Saving model checkpoint to ./results/baselines/VUA/checkpoint-1422
Configuration saved in ./results/baselines/VUA/checkpoint-1422/config.json
Model weights saved in ./results/baselines/VUA/checkpoint-1422/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1638
  Batch size = 32
Saving model checkpoint to ./results/baselines/VUA/ch